<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Изучение-данных-и-их-разделение" data-toc-modified-id="Изучение-данных-и-их-разделение-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Изучение данных и их разделение</a></span></li><li><span><a href="#Построение-моделей" data-toc-modified-id="Построение-моделей-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Построение моделей</a></span><ul class="toc-item"><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Дерево решений</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Проверка-модели-на-тестовой-выборке" data-toc-modified-id="Проверка-модели-на-тестовой-выборке-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Проверка модели на тестовой выборке</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

# Построение модели для определения подходящего тарифа

Оператор мобильной связи выяснил, что многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
В нашем распоряжении предобработанные ранее данные о поведении клиентов, которые уже перешли на эти тарифы. **Цель проекта -  построить модель для задачи классификации, которая выберет подходящий тариф.**

## Изучение данных и их разделение

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import seaborn as sns

In [2]:
df = pd.read_csv('C:\\Users\maria\Desktop\ЯПрактикум\Проекты\Проект6\\users_behavior.csv')

In [3]:
display(df.head())
df.info()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Чтобы обучить и проверить модель необходимо выделить тестовую, валидационную и обучающую выборки в соотношении 3:1:1. Для этого:
- разделим весь Датафрейм целевой признак - столбец `is_ultra` и на признаки - остальные столбцы.
- отделим 20% данных для тестовой выборки
- отделим 25% оставшихся данных для валидационной выборки

In [4]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

In [5]:
# отделяем 20% данных для тестовой выборки
features_train_valid, features_test, target_train_valid, target_test = train_test_split(
    features, target, test_size=0.2, random_state=12345)
# отделяем 25% оставшихся данных для валидационной выборки
features_train, features_valid, target_train, target_valid= train_test_split(
    features_train_valid, target_train_valid, test_size=0.25, random_state=12345)
# для проверки выведем размеры признаков всех выборок
print("Размер признаков для тестовой выборки:", features_test.shape)
print("Размер признаков для валидационной выборки:", features_valid.shape)
print("Размер признаков для обучающей выборки:", features_train
      .shape)

Размер признаков для тестовой выборки: (643, 4)
Размер признаков для валидационной выборки: (643, 4)
Размер признаков для обучающей выборки: (1928, 4)


Данные готовы можно приступать к построению моделей.

## Построение моделей

Так как тарифов только два, перед нами задача бинарной классификации. Поэтому построим три модели, подходящих для этой задачи: дерево решений, случайный лес и логистическую регрессию. Качество модели будем проверять критерием `accuracy`.

### Дерево решений

Для дерева решений возьмём следующие гиперпараметры: `max_depth`, `max_features`,`criterion`, `random_state`, `min_samples_split`, `min_samples_leaf`. В конце кода выведем все лучшие гиперпараметры и качество модели. В модель дерево решений работает достоточно быстро, поэтому переберем сочетания всех этих параметров.

In [6]:
criterions = ['gini', 'entropy'] # список критериев расщепления
max_features = [None, 'auto', 'log2'] # список числа признаков

In [7]:
best_model_1 = None
best_result_1 = 0
best_depth_1 = 0
best_criterion_1 = ''
best_max_feature_1 = ''
best_samples_split_1 = 0
best_samples_leaf_1 = 0

for criterion_1 in criterions:
    for max_feature_1 in max_features:
        for depth_1 in range(1, 10):
            for samples_split_1 in range(10, 200, 10):
                for samples_leaf_1 in range(1, 5):
                    # обучение модели с заданными гиперпараметрами
                    model_1 = DecisionTreeClassifier(random_state=12345, 
                                                     criterion=criterion_1,
                                                     max_features=max_feature_1,
                                                     max_depth=depth_1,
                                                     min_samples_split=samples_split_1, 
                                                     min_samples_leaf=samples_leaf_1)
                    model_1.fit(features_train, target_train)
                    result_1 = accuracy_score(target_valid, model_1.predict(features_valid)) # подсчёт качества модели
                    if result_1 > best_result_1:
                        # запись лучших параметров
                        best_model_1 = model_1 
                        best_result_1 = result_1
                        best_criterion_1 = criterion_1
                        best_max_feature_1 = max_feature_1
                        best_depth_1 = depth_1
                        best_samples_split_1 = samples_split_1
                        best_samples_leaf_1 = samples_leaf_1
                                                
print("Accuracy наилучшей модели на валидационной выборке:", best_result_1, 
      "\nКритерий:", best_criterion_1, 
      "\nКоличество признаков:", best_max_feature_1,
      "\nГлубина дерева:", best_depth_1,
      "\nМинимальное количество примеров:", best_samples_split_1, 
      "\nМинимальное количество объектов:", best_samples_leaf_1)

Accuracy наилучшей модели на валидационной выборке: 0.7962674961119751 
Критерий: entropy 
Количество признаков: auto 
Глубина дерева: 9 
Минимальное количество примеров: 10 
Минимальное количество объектов: 4


### Случайный лес

Для случайного леса используем гиперпараметры: `n_estimators`, `max_depth`, `max_features`,`criterion`, `random_state`. Так как данная модель и так тратит больше времени на обучение, гиперпараметры `min_samples_split` и `min_samples_leaf` опустим.

In [8]:
best_model_2 = None
best_result_2 = 0
best_est_2 = 0
best_depth_2 = 0
best_criterion_2 = ''
best_max_feature_2 = ''

for criterion_2 in criterions:
    for max_feature_2 in max_features:
        for est_2 in range(1, 15):
            for depth_2 in range(1, 10):
                        # обучение модели с заданными гиперпараметрами
                        model_2 = RandomForestClassifier(random_state=12345,
                                                         criterion=criterion_2,
                                                         max_features=max_feature_2,
                                                         n_estimators=est_2,
                                                         max_depth=depth_2)
                        model_2.fit(features_train, target_train)
                        result_2 = accuracy_score(target_valid, model_2.predict(features_valid)) # подсчёт качества модели
                        if result_2 > best_result_2:
                            # запись лучших параметров
                            best_model_2 = model_2
                            best_result_2 = result_2
                            best_criterion_2 = criterion_2
                            best_max_feature_2 = max_feature_2
                            best_est_2 = est_2
                            best_depth_2 = depth_2
                            
print("Accuracy наилучшей модели на валидационной выборке:", best_result_2, 
      "\nКритерий:", best_criterion_2, 
      "\nКоличество признаков:", best_max_feature_2,
      "\nКоличество деревьев:", best_est_2,
      "\nГлубина дерева:", best_depth_2)

Accuracy наилучшей модели на валидационной выборке: 0.7993779160186625 
Критерий: gini 
Количество признаков: None 
Количество деревьев: 10 
Глубина дерева: 9


### Логистическая регрессия

Для данной модели возьмем гиперпараметры `random_state`, `solver`, `max_iter` и зададим им сразу значения.

In [9]:
model_3 = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model_3.fit(features_train, target_train)
result_3 = model_3.score(features_valid, target_valid)
        
print("Accuracy наилучшей модели на валидационной выборке:", result_3)

Accuracy наилучшей модели на валидационной выборке: 0.7262830482115086


### Вывод

- Значения accuracy моделей Дерево решений (0.796) и Случайный лес (0.799) получились очень близки, модель Логистической регрессии им явно уступает (0.726). 
- В модели Дерево решений использовалось больше гиперпараметров, чем в Случайном лесе, и получилось, что время выполнения задачи почти одинаковое. Быстрее всего явно справляется Логистическая регрессия, но это связано с тем, что в этой модели гиперпараметры не перебирались.
- Так как нам важно именно значение accuracy в данному случае, наилучшей моделью будет являться Случайный лес.

## Проверка модели на тестовой выборке

In [10]:
final_result = accuracy_score(target_test, best_model_2.predict(features_test))
print("Accuracy наилучшей модели на тестовой выборке:", final_result)

Accuracy наилучшей модели на тестовой выборке: 0.7931570762052877


Получили нужное значение Accuracy равное 0.79 на тестовой выборке.

## Общий вывод

- Для обучения моделей был выбран целевой признак - наличие тарифа ultra (1-этот тариф, 0 - другой). Все остальные столбцы подошли для признаков.
- Все данные были разбиты на тестовую, валидационную и обучающую выборки в соотношении 1:1:3.
- Для обучения были выбраны три модели: Дерево решений, Случайный лес, Логистическая регрессия.
- Для модели Дерево решений взяли и перебрали гиперпараметры: `max_depth`, `max_features`,`criterion`, `random_state`, `min_samples_split`, `min_samples_leaf`.
- Для модели Случайного леса взяли и перебрали гиперпараметры:`n_estimators`, `max_depth`, `max_features`,`criterion`, `random_state`.
- Для модели Логистической регресии взяли заданные гиперпараметры `random_state`, `solver`, `max_iter`.
- Лучшее значение accuracy - 0.799 на валидационной выборке получилось в модели Случайный лес. При проверке этой модели на тестовой выборке получили также достаточно высокое значение accuracy - 0.793.